In [1]:
import findspark
findspark.init('/opt/spark-3.4.3/')

import pyspark

from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("demo").getOrCreate()

https://repos.spark-packages.org/ added as a remote repository with the name: repo-1
Ivy Default Cache set to: /data/home/lyudmila.chapligina/.ivy2/cache
The jars for the packages stored in: /data/home/lyudmila.chapligina/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-12a145f5-b9cc-4628-b917-d78e0d7240a0;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/spark-3.4.3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.3 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.5 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 423ms :: artifacts dl 12ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.apache.commons#commons-pool2;2.11.1 from central in [default]
	org.apache.hadoop#hadoop-client-api;3.3.4 from central in [default]
	org.apache.hadoop#

In [3]:
# путь к файлу
file_path = "lab01/u.data"

# чтение файла вручную
rows = []
with open(file_path, 'r') as file:
    for line in file:
        user_id, item_id, rating, timestamp = map(int, line.strip().split('\t'))
        rows.append((rating, item_id)) # отбираем только нужные столбцы

In [4]:
# cоздание RDD из списка кортежей
rdd = spark.sparkContext.parallelize(rows)

In [5]:
def count_rating(rdd): # cчитаем кол-во оценок для всех фильмов
    hist = rdd\
    .map(lambda pair: (pair[0], 1))\
    .reduceByKey(lambda x, y: x + y)\
    .collect()
    
    hist = sorted(hist, key=lambda x: x[0]) # cортируем по оценкам 1-5
    
    return [x[1] for x in hist] # последовательность количества оценок 1-5

In [6]:
rdd_318 = rdd.filter(lambda x: x[1]==318) # фильтруем датасет по id фильма как в задании

In [7]:
# создаем структуру для записи в json
lab01_dict = {
    "hist_film": count_rating(rdd_318),
    "hist_all": count_rating(rdd)
}

In [8]:
# записываем в json
output = "lab01/lab01.json"
with open(output, "w") as f:
    json.dump(lab01_dict, f, indent=4)